# Adult Dataset MLP Training - mk5
#### 16 May 2022

## Load Dependencies <a id="libraries"></a>

In [1]:
import pandas as pd
import numpy as np
import timeit
import os
import math

In [2]:
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
import warnings
warnings.filterwarnings('ignore')

### Notebook options and parameters

In [5]:
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250

In [6]:
# random seed for consistency in pytorch and numpy
SEED = 23

torch.manual_seed(SEED)
np.random.seed(SEED)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Hyperparameter options

In [8]:
# activation functions
elu = nn.ELU()
leaky = nn.LeakyReLU()
relu = nn.ReLU()
tanh = nn.Tanh()
sigmoid = nn.Sigmoid()

# loss functions
mse = nn.MSELoss()
bce = nn.BCELoss()
kld = nn.KLDivLoss()

# optimizers
adam = torch.optim.Adam
sgd = torch.optim.SGD

## Custom Functions <a id="functions"></a>

#### Create dataset for DataLoader

In [9]:
class dataset(Dataset):
    def __init__(self, data, target):
        self.X = torch.tensor(data, dtype=torch.float32, device=device)
        self.y = torch.tensor(target, dtype=torch.float32, device=device)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

#### Training function

In [10]:
def train(data, model, criterion, metric, optimizer):
    size = len(data.dataset)
    batches = len(data)
    running_loss = []
    acc = 0.0
    correct = 0
    predictions = []
    actuals = []
    loss = 0.0
    acc = 0.0
    current = 0
    
    for batch, (X, y) in enumerate(data,1):
        
        # forward step
        optimizer.zero_grad() # zero out the gradients
        y_hat = model(X)  #forward pass
        
        y = y.unsqueeze(-1) # reduce dimensions of tensor from [256, 1] to [256]
        
        loss = criterion(y_hat, y)  # calculate loss
       
        #  backprop step
        loss.backward()       # backward pass through model; computes gradients
        optimizer.step()      # update weights
        
        # metrics
        y = y.int()
        acc = metric(y_hat, y)

        y_hat = y_hat.cpu().detach().numpy()
        y_hat = y_hat.reshape(-1)
        
        y = y.cpu().detach().numpy()
        y = y.reshape(-1)
        
        predictions.append(y_hat)
        actuals.append(y)
            
        running_loss.append(loss.item())
    
        ## every 50 batches, store the results
        #if batch % 50 == 0:
        #    current = batch * len(X)
        
    print(f"\tTraining\tAccuracy: {acc:1.10f}\tLoss: {loss:1.10f}")
    
    return predictions, running_loss

#### Testing function

In [11]:
def test(data, model, criterion, metric):
    size = len(data.dataset)
    batches = len(data)
    tst_loss = 0
    correct = 0
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X, y in data:
            y_hat = model(X)
            y = y.unsqueeze(-1)
            tst_loss += criterion(y_hat, y).item()
            
            y = y.int()
            acc = metric(y_hat, y)
            
            y_hat = y_hat.cpu().detach().numpy()
            y_hat = y_hat.reshape(-1)
        
            y = y.cpu().detach().numpy()
            y = y.reshape(-1)
        
            predictions.append(y_hat)
            actuals.append(y)
            
    tst_loss /= batches
    correct /= size
    
    print(f"\tTest error\tAccuracy: {acc:1.10f}\tLoss: {tst_loss:1.10f}")
    return predictions, tst_loss, acc

#### Evaluation - train and test

In [12]:
def evaluate(training_data, test_data, model, loss_function, metric, optimizer, epochs, early_stop=True):
    training_results = []
    training_actuals = []
    training_losses = []

    testing_results = []
    testing_actuals = []

    start = timeit.default_timer()
    
    # early stop criteria
    
    default_patience = 3
    patience = default_patience
    running_loss = []
    running_acc = []

    for epoch in range(1,epochs+1):
        print (f"Epoch {epoch} / {epochs}")

        trng_result, trng_loss = train(training_data, model, loss_function, metric, optimizer)
        tst_result, tst_loss, acc = test(test_data, model, loss_function, metric)
        
        training_results.append(trng_result)
        training_losses.append(trng_loss)
        testing_results.append(tst_result)
        running_loss.append(tst_loss)
        running_acc.append(acc)
        
        # check for early stop
        if epoch > 3:            
            if early_stop and ((tst_loss >= running_loss[-2]) or math.isclose(acc,running_acc[-2], abs_tol=1e-8)):
                patience = patience - 1
            elif early_stop and (tst_loss < running_loss[-2]):
                patience = default_patience

            if early_stop and (patience <= 0):
                print("Early stop - ", end="")
                break

    
    print("Finished")
    print("* " * 30)
    stop = timeit.default_timer()
    print(f"Execution time (in seconds): {stop - start}")

    return training_results, training_losses, testing_results

#### Multilayer Perception (MLP)

In [13]:
class MLP(nn.Module):
    
    def __init__(self, in_size, hidden1=relu, hidden2=relu, hidden3=relu, out_layer=sigmoid):
        super(MLP, self).__init__()
        
        # construct model dependent on size of inputs
        out1_2 = 2*in_size // 3
        out2_3 = 2*out1_2 // 3
        out3_4 = 2*out2_3 // 3

        self.linear_stack = nn.Sequential(
            nn.Linear(in_size, out1_2),  # input layer
            hidden1,
            nn.Linear(out1_2, out2_3),   # hidden layer
            hidden2,
            nn.Linear(out2_3, out3_4),   # hidden layer
            hidden3,
            nn.Linear(out3_4, 1),        # output layer
            out_layer
        )
        
    def forward(self, x):
        out = self.linear_stack(x)

        return out

#### Flatten function

In [14]:
def flatten(thing):
    '''
    thing: a list
    
    flatten receives a multi-level list and flattens it down 2 layers
    '''
    #thing = [element for sublist in thing for element in sublist]
    return [element for sublist in thing for element in sublist]

#### Prepare data for eval and analysis

In [15]:
def prep(name, training_results, testing_results):
    trng_preds = [[] for _ in range(len(training_results))]

    tst_preds = [[] for _ in range(len(testing_results))]
    
    for i in range(len(training_results)):
        trng_preds[i] = flatten(training_results[i])

        tst_preds[i] = flatten(testing_results[i])

    return trng_preds, tst_preds

#### Send results to tensorboard

In [16]:
def to_tensorboard(tailend, directory, dataset, train_preds, test_preds, y_train, y_test):
    acc_trng_relu = []
    acc_tst_relu = []
    loss_relu = []

    writer = SummaryWriter(log_dir=directory)

    for i in range(len(train_preds)):
        y_hat = np.array(np.round(train_preds[i]))
        y_hat_tst = np.array(np.round(test_preds[i]))
        trng_acc = (y_hat == y_train).sum()/len(train_preds[i])
        test_acc = (y_hat_tst == y_test).sum()/len(test_preds[i])

        avg_loss = np.average(training_losses[i])

        acc_trng_relu.append(trng_acc)
        acc_tst_relu.append(test_acc)
        loss_relu.append(avg_loss)
        writer.add_scalar(f"Training Accuracy {tailend}", trng_acc, i)
        writer.add_scalar(f"Test Accuracy {tailend}", test_acc, i)
        writer.add_scalar(f"Training Loss {tailend}", avg_loss, i)
        writer.add_pr_curve(f"Precision-Recall Curve {tailend}", y_test, y_hat_tst, i)

    t, _ = next(iter(dataset))
    writer.add_graph(model, t)

    writer.close()

#### Custom save

In [17]:
def custom_save(name, data, kind=1):
    '''
    name : string
        designated filename
    data : data or pytorch model
        the data to save
    kind : int
        sentinel value - 1 if pytorch model, 0 otherwise
    
    custom_save stores the data passed into the function into a file with the provided name
    '''
    
    if kind == 1:
        ex = ".pth"
    else:
        ex = ".parquet"
    
    sentinel = True
    i = 1

    while sentinel:
        dirlist = os.listdir()

        if name not in dirlist:
            if kind == 1:
                torch.save(data, name)
            else:
                data.to_parquet(name)
            print(f"{name} has been saved.")                
            sentinel = False
        if name in dirlist:
            print(f"{name} already exists.", end=" ")
            temp, ext = name.split(ex)
            if "_v" in temp:
                temp, _ = temp.split("_v")
            name = f"{temp}_v{i}{ex}"
            i = i + 1
            print(f"Changing file name to: {name}")

# Import Data<a id="data"></a>

#### Uncomment next when using google colab

In [18]:
#data_train = pd.read_parquet("https://github.com/ollin23/bias/blob/main/adult_train_mk2.parquet?raw=true")
#data_test = pd.read_parquet("https://github.com/ollin23/bias/blob/main/adult_test_mk2.parquet?raw=true")

#### Use the following cell if the data is stored locally

In [19]:
data_train = pd.read_parquet("adult_train_mk2.parquet")
data_test = pd.read_parquet("adult_test_mk2.parquet")

#### Peak the training data

In [20]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Columns: 108 entries, age to target
dtypes: int64(7), uint8(101)
memory usage: 4.9 MB


In [21]:
data_train.shape

(32561, 108)

In [22]:
data_train.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,gender,native-country_ ?,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,target
0,39,77516,13,2174,0,40,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,50,83311,13,0,0,13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,38,215646,9,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,53,234721,7,0,0,40,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,28,338409,13,0,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
list(data_train.columns)

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_ ?',
 'workclass_ Federal-gov',
 'workclass_ Local-gov',
 'workclass_ Never-worked',
 'workclass_ Private',
 'workclass_ Self-emp-inc',
 'workclass_ Self-emp-not-inc',
 'workclass_ State-gov',
 'workclass_ Without-pay',
 'education_ 10th',
 'education_ 11th',
 'education_ 12th',
 'education_ 1st-4th',
 'education_ 5th-6th',
 'education_ 7th-8th',
 'education_ 9th',
 'education_ Assoc-acdm',
 'education_ Assoc-voc',
 'education_ Bachelors',
 'education_ Doctorate',
 'education_ HS-grad',
 'education_ Masters',
 'education_ Preschool',
 'education_ Prof-school',
 'education_ Some-college',
 'marital-status_ Divorced',
 'marital-status_ Married-AF-spouse',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Married-spouse-absent',
 'marital-status_ Never-married',
 'marital-status_ Separated',
 'marital-status_ Widowed',
 'occupation_ ?',
 'occupation_ Adm-clerical',
 'occupation_ Ar

#### Peak the test data

In [24]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 1 to 16281
Columns: 107 entries, age to target
dtypes: float64(5), int64(2), uint8(100)
memory usage: 2.4 MB


In [25]:
data_test.shape

(16281, 107)

In [26]:
data_test.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,gender,native-country_ ?,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,target
1,25,226802.0,7.0,0.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,38,89814.0,9.0,0.0,0.0,50.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,28,336951.0,12.0,0.0,0.0,40.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,44,160323.0,10.0,7688.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,18,103497.0,10.0,0.0,0.0,30.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [27]:
list(data_test.columns)

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_ ?',
 'workclass_ Federal-gov',
 'workclass_ Local-gov',
 'workclass_ Never-worked',
 'workclass_ Private',
 'workclass_ Self-emp-inc',
 'workclass_ Self-emp-not-inc',
 'workclass_ State-gov',
 'workclass_ Without-pay',
 'education_ 10th',
 'education_ 11th',
 'education_ 12th',
 'education_ 1st-4th',
 'education_ 5th-6th',
 'education_ 7th-8th',
 'education_ 9th',
 'education_ Assoc-acdm',
 'education_ Assoc-voc',
 'education_ Bachelors',
 'education_ Doctorate',
 'education_ HS-grad',
 'education_ Masters',
 'education_ Preschool',
 'education_ Prof-school',
 'education_ Some-college',
 'marital-status_ Divorced',
 'marital-status_ Married-AF-spouse',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Married-spouse-absent',
 'marital-status_ Never-married',
 'marital-status_ Separated',
 'marital-status_ Widowed',
 'occupation_ ?',
 'occupation_ Adm-clerical',
 'occupation_ Ar

####  The dataframe shapes are different, find out what feature is missing  from the test set

In [28]:
missing = set(list(data_train.columns)) - set(list(data_test.columns))
missing

{'native-country_ Holand-Netherlands'}

#### Add column

In [29]:
i = list(data_train.columns).index(list(missing)[0])
i

80

In [30]:
data_test.insert(i, list(missing)[0], 0)

In [31]:
data_test.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,gender,native-country_ ?,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,target
1,25,226802.0,7.0,0.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,38,89814.0,9.0,0.0,0.0,50.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,28,336951.0,12.0,0.0,0.0,40.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,44,160323.0,10.0,7688.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,18,103497.0,10.0,0.0,0.0,30.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### Process Data for Neural Net Usage <a id="convert"></a>

#### Split features into X and y

In [32]:
X_train = data_train.drop("target", axis=1).values.astype(np.float32)
y_train = data_train.target.values.astype(np.float32)

In [33]:
X_test = data_test.drop("target", axis=1).values.astype(np.float32)
y_test = data_test.target.values.astype(np.float32)

#### Standardize data

In [34]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

#### Split into training and testing sets

In [35]:
X_train_scaled.shape

(32561, 107)

In [36]:
X_test_scaled.shape

(16281, 107)

#### Make datasets

In [37]:
scaled_train = dataset(X_train_scaled, y_train)
scaled_test = dataset(X_test_scaled, y_test)

##### Construct test results dataframe for persistent storage

In [38]:
features = list(data_train.drop("target", axis=1).columns)

#temp = np.round(scaler.inverse_transform(X_test)).astype(int)
df = pd.DataFrame(X_test, columns=features)
df["target"] = y_test

##### Note
For this dataset: Female == 1, Male == 0

# Experiment Phase 2 - Adult Data <a id="experiments"></a>

## Trial 1 - ReLU (Loans) <a id="relu"></a>

### Preliminaries

In [39]:
suffix = "mk5"
directory = f"AdultIncome_{suffix}"
prefix = "adult"

#### Hyperparameters <a id="hyperparameters"></a>

In [40]:
alpha = 1e-5 # 0.0001
epochs = 150
batches = 100
hidden1 = relu
hidden2 = relu
hidden3 = relu
out_layer = sigmoid

#### DataLoader

In [41]:
# Helper function to feed data into pytorch neural network, only needs to execute once
training_s = DataLoader(scaled_train, batch_size=batches)
test_s = DataLoader(scaled_test, batch_size=batches)

#### Construct Model

In [43]:
# instantiate model
model = MLP(X_train_scaled.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=107, out_features=71, bias=True)
    (1): ReLU()
    (2): Linear(in_features=71, out_features=47, bias=True)
    (3): ReLU()
    (4): Linear(in_features=47, out_features=31, bias=True)
    (5): ReLU()
    (6): Linear(in_features=31, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [44]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=True)

Epoch 1 / 150
	Training	Accuracy: 0.2622950673	Loss: 0.2720525563
	Test error	Accuracy: 0.0123456791	Loss: 0.2938973099
Epoch 2 / 150
	Training	Accuracy: 0.2459016442	Loss: 0.2633739114
	Test error	Accuracy: 0.0370370373	Loss: 0.2779112881
Epoch 3 / 150
	Training	Accuracy: 0.3606557250	Loss: 0.2526195943
	Test error	Accuracy: 0.1975308657	Loss: 0.2584491890
Epoch 4 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.2378005534
	Test error	Accuracy: 0.7407407165	Loss: 0.2327621099
Epoch 5 / 150
	Training	Accuracy: 0.7704917789	Loss: 0.2196252048
	Test error	Accuracy: 0.9135802388	Loss: 0.2025433414
Epoch 6 / 150
	Training	Accuracy: 0.7540983558	Loss: 0.2006962448
	Test error	Accuracy: 0.9753086567	Loss: 0.1710744425
Epoch 7 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.1837757081
	Test error	Accuracy: 0.9876543283	Loss: 0.1423216699
Epoch 8 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.1707754135
	Test error	Accuracy: 1.0000000000	Loss: 0.1196570161
Epoch 9 / 150
	Training	Accuracy: 0.7377

### Prepare results for analysis

In [45]:
tailend = f"ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save ReLU Metrics to Tensorboard <a id="relu-metrics-for-tensorboard"></a>

In [46]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ReLU predictions to persistent storage

In [47]:
df["pred_relu"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_relu_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_relu_scaled.pth")

adult_relu_mk5.pth has been saved.


## Trial 2 - Tanh (Loans) <a id="tanh"></a>

#### Hyperparameters

In [48]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = tanh
hidden2 = tanh
hidden3 = tanh
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [49]:
# instantiate model
model = MLP(X_train_scaled.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=107, out_features=71, bias=True)
    (1): Tanh()
    (2): Linear(in_features=71, out_features=47, bias=True)
    (3): Tanh()
    (4): Linear(in_features=47, out_features=31, bias=True)
    (5): Tanh()
    (6): Linear(in_features=31, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Training and test

In [50]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.6065573692	Loss: 0.2471694350
	Test error	Accuracy: 0.3086419702	Loss: 0.2527910550
Epoch 2 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.2316676527
	Test error	Accuracy: 0.6049382687	Loss: 0.2379834593
Epoch 3 / 150
	Training	Accuracy: 0.7868852615	Loss: 0.2128355503
	Test error	Accuracy: 0.6913580298	Loss: 0.2237543123
Epoch 4 / 150
	Training	Accuracy: 0.8032786846	Loss: 0.1927276850
	Test error	Accuracy: 0.6790123582	Loss: 0.2119828731
Epoch 5 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1750514954
	Test error	Accuracy: 0.6790123582	Loss: 0.2036232214
Epoch 6 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1613581181
	Test error	Accuracy: 0.7037037015	Loss: 0.1972567116
Epoch 7 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1511050612
	Test error	Accuracy: 0.7037037015	Loss: 0.1911034566
Epoch 8 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1433504224
	Test error	Accuracy: 0.7283950448	Loss: 0.1844227319
Epoch 9 / 150
	Training	Accuracy: 0.8196

#### Prepare tanh results for analysis

In [51]:
tailend = f"Tanh ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Generate tensorboard summary

In [52]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Store Tanh results in dataframe 

In [53]:
df["pred_tanh"] = np.array(np.round(test_preds[-1])).astype(int)

name = f"{prefix}_tanh_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_tanh_scaled.pth")

adult_tanh_mk5.pth has been saved.


## Trial 3 - ELU (Loans)<a id="elu"></a>

#### Hyperparameters

In [54]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = elu
hidden2 = elu
hidden3 = elu
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [55]:
# instantiate model
model = MLP(X_train_scaled.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=107, out_features=71, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=71, out_features=47, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=47, out_features=31, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=31, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [56]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.2200410217
	Test error	Accuracy: 1.0000000000	Loss: 0.1908182355
Epoch 2 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.2088468969
	Test error	Accuracy: 1.0000000000	Loss: 0.1809645440
Epoch 3 / 150
	Training	Accuracy: 0.7704917789	Loss: 0.1937147230
	Test error	Accuracy: 0.9506173134	Loss: 0.1716122784
Epoch 4 / 150
	Training	Accuracy: 0.8032786846	Loss: 0.1755848527
	Test error	Accuracy: 0.8148148060	Loss: 0.1648287889
Epoch 5 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1584289074
	Test error	Accuracy: 0.7777777910	Loss: 0.1619630710
Epoch 6 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1450964957
	Test error	Accuracy: 0.7777777910	Loss: 0.1610923773
Epoch 7 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1354373544
	Test error	Accuracy: 0.7654321194	Loss: 0.1595623097
Epoch 8 / 150
	Training	Accuracy: 0.8196721077	Loss: 0.1283674836
	Test error	Accuracy: 0.7530864477	Loss: 0.1566300131
Epoch 9 / 150
	Training	Accuracy: 0.8032

#### Prepare ELU results for analysis

In [57]:
tailend = f"ELU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save metrics to tensorboad

In [58]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ELU test results to persistent storage

In [59]:
df["pred_elu"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_elu_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_elu_scaled.pth")

adult_elu_mk5.pth has been saved.


## Trial 4 - Leaky ReLU (Loans) <a id="leaky"></a>

#### Hyperparameters

In [60]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = leaky
hidden2 = leaky
hidden3 = leaky
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct Model

In [61]:
# instantiate model
model = MLP(X_train_scaled.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=107, out_features=71, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=71, out_features=47, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=47, out_features=31, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=31, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [62]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs)

Epoch 1 / 150
	Training	Accuracy: 0.7540983558	Loss: 0.2417876124
	Test error	Accuracy: 1.0000000000	Loss: 0.2347400084
Epoch 2 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.2293746024
	Test error	Accuracy: 1.0000000000	Loss: 0.2128053582
Epoch 3 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.2118215561
	Test error	Accuracy: 1.0000000000	Loss: 0.1828154615
Epoch 4 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.1920461953
	Test error	Accuracy: 1.0000000000	Loss: 0.1491942195
Epoch 5 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.1754877269
	Test error	Accuracy: 1.0000000000	Loss: 0.1200221974
Epoch 6 / 150
	Training	Accuracy: 0.7377049327	Loss: 0.1640998870
	Test error	Accuracy: 1.0000000000	Loss: 0.0992250325
Early stop - Finished
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
Execution time (in seconds): 5.201195111963898


#### Prepare Leaky ReLU data for analysis

In [63]:
tailend = f"Leaky ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Add Leaky ReLU data to persistent storage

In [64]:
df["pred_leaky"] = np.array(np.round(test_preds[-1])).astype(int)

name = f"{prefix}_leaky_{suffix}.pth"
custom_save(name, model)

#torch.save(model, "loans_leaky_scaled.pth")

adult_leaky_mk5.pth has been saved.


# Results

In [65]:
df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,gender,native-country_ ?,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,target,pred_relu,pred_tanh,pred_elu,pred_leaky
0,25.0,226802.0,7.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0
1,38.0,89814.0,9.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0
2,28.0,336951.0,12.0,0.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0
3,44.0,160323.0,10.0,7688.0,0.0,40.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1

## Save models

In [66]:
name = f"{prefix}_results_{suffix}.parquet"
custom_save(name, df, 0)

adult_results_mk5.parquet has been saved.
